In [ ]:
# SEL 1: Instalasi Library
# Jalankan sel ini terlebih dahulu
!pip install streamlit pyngrok joblib scikit-learn tensorflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.2 MB/s eta 0:00:00


In [ ]:
# SEL 2: Menulis File app.py
# Pastikan '%%writefile app.py' adalah baris PERTAMA di sel ini
%%writefile app.py

import streamlit as st
import joblib
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os

# =============================================================================
# KONFIGURASI PENTING (SESUAIKAN DENGAN TRAINING ANDA)
# =============================================================================

# TODO: Sesuaikan nama kelas ini. Urutannya HARUS SAMA PERSIS
# dengan urutan kelas saat Anda melatih model.
# Contoh: 0=Glioma, 1=Meningioma, 2=No Tumor, 3=Pituitary
CLASS_NAMES = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary'] # <--- SESUAIKAN INI

# TODO: Sesuaikan ukuran input gambar jika berbeda
IMG_HEIGHT = 224
IMG_WIDTH = 224

# Path ke model Anda (pastikan sudah di-upload ke Colab)
RF_MODEL_PATH = r'/content/random_forest_model.pkl'
XGB_MODEL_PATH = r'/content/xgboost_model.pkl'

# =============================================================================
# Fungsi Helper (Memuat Model & Ekstraksi Fitur)
# =============================================================================

@st.cache_resource
def load_all_models():
    """Memuat model RF, XGB, dan MobileNetV2 sekali saja."""
    try:
        rf_model = joblib.load(RF_MODEL_PATH)
    except FileNotFoundError:
        st.error(f"Error: File model Random Forest '{RF_MODEL_PATH}' tidak ditemukan.")
        rf_model = None
    except Exception as e:
        st.error(f"Error saat memuat model Random Forest: {e}")
        rf_model = None

    try:
        xgb_model = joblib.load(XGB_MODEL_PATH)
    except FileNotFoundError:
        st.error(f"Error: File model XGBoost '{XGB_MODEL_PATH}' tidak ditemukan.")
        xgb_model = None
    except Exception as e:
        st.error(f"Error saat memuat model XGBoost: {e}")
        xgb_model = None

    # Muat MobileNetV2 untuk ekstraksi fitur
    base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
    base_model.trainable = False

    return rf_model, xgb_model, base_model

def extract_features(img, base_model):
    """
    Menerima PIL Image, mengubahnya, dan mengekstrak fitur menggunakan MobileNetV2.
    """
    if img.mode != 'RGB':
        img = img.convert('RGB')

    img_resized = img.resize((IMG_HEIGHT, IMG_WIDTH))
    img_array = keras_image.img_to_array(img_resized)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_expanded)
    features = base_model.predict(img_preprocessed)

    return features

# =============================================================================
# Layout Aplikasi Streamlit
# =============================================================================

st.set_page_config(page_title="Analisis Tumor Otak", layout="wide")
st.title("🧠 Analisis Citra MRI untuk Deteksi Tumor Otak")
st.markdown("Perbandingan Model Klasifikasi: **Random Forest** vs **XGBoost**")

# Memuat semua model
rf_model, xgb_model, feature_extractor = load_all_models()

models_loaded = rf_model is not None and xgb_model is not None

if models_loaded:
    st.success("Model Random Forest, XGBoost, dan MobileNetV2 berhasil dimuat.")
else:
    st.error("Satu atau lebih model gagal dimuat. Pastikan file 'random_forest_model.pkl' dan 'xgboost_model.pkl' sudah di-upload.")

st.markdown("---")

uploaded_file = st.file_uploader("Unggah Citra MRI (format: .jpg, .jpeg, .png)", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Citra MRI yang Diunggah", width=300)

    st.markdown("---")

    classify_button = st.button("Mulai Analisis Sekarang", disabled=not models_loaded)

    if classify_button:
        if not models_loaded:
            st.error("Analisis tidak dapat dilanjutkan karena model gagal dimuat.")
        else:
            with st.spinner("Sedang menganalisis... (Ekstraksi fitur & Prediksi)"):

                # 1. Ekstraksi Fitur
                features = extract_features(image, feature_extractor)

                # 2. Prediksi menggunakan Random Forest
                try:
                    rf_pred_proba = rf_model.predict_proba(features)
                    rf_confidence_score = np.max(rf_pred_proba) * 100
                    rf_class_index = np.argmax(rf_pred_proba)
                    rf_prediction = CLASS_NAMES[rf_class_index]
                except Exception as e:
                    st.error(f"Error prediksi Random Forest: {e}")
                    rf_prediction, rf_confidence_score = "Error", 0

                # 3. Prediksi menggunakan XGBoost
                try:
                    xgb_pred_proba = xgb_model.predict_proba(features)
                    xgb_confidence_score = np.max(xgb_pred_proba) * 100
                    xgb_class_index = np.argmax(xgb_pred_proba)
                    xgb_prediction = CLASS_NAMES[xgb_class_index]
                except Exception as e:
                    st.error(f"Error prediksi XGBoost: {e}")
                    xgb_prediction, xgb_confidence_score = "Error", 0

                # 4. Tampilkan hasil berdampingan
                st.subheader("Hasil Perbandingan Prediksi")

                col1, col2 = st.columns(2)

                with col1:
                    st.metric(
                        label="Random Forest 🌳",
                        value=rf_prediction,
                        delta=f"Confidence: {rf_confidence_score:.2f}%"
                    )

                with col2:
                    st.metric(
                        label="XGBoost 🚀",
                        value=xgb_prediction,
                        delta=f"Confidence: {xgb_confidence_score:.2f}%"
                    )

else:
    st.info("Silakan unggah gambar MRI untuk memulai analisis.")

Overwriting app.py


In [ ]:
# SEL 3: Menjalankan Streamlit dengan Ngrok
# Ganti <TOKEN_AUTH_NGROK_ANDA> dengan token Anda
from pyngrok import ngrok
import os

# Masukkan Authtoken Ngrok Anda di sini
AUTHTOKEN = "2zqw3dtSEzhJGnNXOmBPnPHbdaZ_2yZ811k2wM3fdHx1WKx3W"  # <-- GANTI DENGAN TOKEN ANDA
os.system(f"ngrok config add-authtoken {AUTHTOKEN}")

# Buka tunnel ke port 8501 (port default Streamlit)
public_url = ngrok.connect(8501)
print(f"URL Publik Streamlit Anda: {public_url}")

# Jalankan aplikasi Streamlit
!streamlit run app.py

URL Publik Streamlit Anda: NgrokTunnel: "https://3a78609005ad.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.201.81:8501

2025-10-24 04:20:16.997610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761279617.024878    1431 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761279617.032946    1431 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761279617.052768    1431 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same targe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')